## 23년 3분기 장바구니 분석

### 1. 라이브러리 임포트 및 설정

In [ ]:
import os
import re
import numpy as np
import pandas as pd

# 장바구니 분석 라이브러리
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

#### 1.1. 파일 경로 설정
분석에 필요한 파일들의 경로

In [ ]:
RAW_DATA_DIR = './raw'
CATEGORY_REGEX_PATH = './category_regex/category_regex.json'
OTHER_SKU_PATH = './category_regex/other_sku.json'

### 2. 기능별 함수 정의
분석 과정을 단계별로 함수화하여 코드의 가독성과 재사용성을 향상

In [ ]:
def load_raw_data(directory: str) -> pd.DataFrame:
    """
    지정된 디렉토리에서 'ken_basket_analysis_'로 시작하는 모든 zip 파일을 로드하여
    하나의 데이터프레임으로 결합
    """
    df_list = []
    print(f"'{directory}' 디렉토리에서 데이터 로딩 중...")
    for filename in os.listdir(directory):
        if filename.startswith('ken_basket_analysis_') and filename.endswith('.zip'):
            file_path = os.path.join(directory, filename)
            # zip 압축 파일을 바로 DataFrame으로
            df = pd.read_csv(file_path, compression='zip')
            df_list.append(df)
    
    if not df_list:
        raise FileNotFoundError(f"해당 경로에 분석할 파일이 없습니다: {directory}")
        
    # 로드된 모든 데이터프레임을 하나로 통합합
    combined_df = pd.concat(df_list, ignore_index=True)
    print(f"데이터 로딩 완료. 총 {len(combined_df):,} 행.")
    return combined_df

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    원본 데이터프레임을 분석에 적합한 형태로 전처리합니다.
    - 컬럼 이름 변경 (영문 소문자)
    - 'date' 컬럼을 datetime 형식으로 변환
    - 유효한 주문 필터링 (모델코드, 주문, 매출액 정보가 모두 있는 경우)
    """
    # 원본 수정을 방지하기 위해 데이터프레임 복사
    proc_df = df.copy()
    proc_df.columns = ['date', 'model_code', 'order', 'revenue']
    proc_df['date'] = pd.to_datetime(proc_df['date'])
    
    # 유효한 주문 조건 정의
    is_valid_order = (
        proc_df['model_code'].notna() &
        (proc_df['order'] > 0) &
        (proc_df['revenue'] > 0)
    )
    
    # 조건에 맞는 데이터만 필터링하고 날짜순으로 정렬
    proc_df = proc_df[is_valid_order].sort_values('date').reset_index(drop=True)
    return proc_df

def split_model_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    'model_code'에 쉼표로 구분된 여러 코드가 있는 경우, 이를 개별 행으로 분리리
    하나의 주문에 여러 상품이 포함된 경우를 정확히 분석
    """
    # 원본 데이터의 index를 'order_id'로 사용하기 위해 유지
    df_indexed = df.reset_index().rename(columns={'index': 'order_id'})
    
    # 쉼표를 기준으로 model_code를 분리하고, explode로 여러 행으로
    split_df = df_indexed.assign(
        model_code=df_indexed['model_code'].str.split(',')
    ).explode('model_code')
    
    # 분리된 모델코드의 좌우 공백을 제거
    split_df['model_code'] = split_df['model_code'].str.strip()
    
    return split_df.reset_index(drop=True)

def categorize_model_codes(unique_model_codes: pd.Series, regex_df: pd.DataFrame, sku_df: pd.DataFrame) -> pd.DataFrame:
    """
    정규식과 별도 SKU 리스트를 사용하여 모델 코드를 효율적으로 분류
    - 1차: 정규식 기반 분류
    - 2차: 1차에서 미분류된 코드를 SKU 리스트(JSON) 기반으로 분류
    """
    # 분류 결과를 저장할 데이터프레임 생성
    cat_df = pd.DataFrame(unique_model_codes, columns=['model_code'])
    cat_df[['div', 'category1', 'category2']] = np.nan

    # 1. 정규식을 사용한 분류 (벡터화 연산으로 성능 개선)
    print("1차 분류(정규식) 시작...")
    for _, row in regex_df.iterrows():
        # 아직 분류되지 않은 코드에 대해서만 정규식 매칭
        mask = cat_df['category1'].isna() & cat_df['model_code'].str.contains(row['regex'], na=False, regex=True)
        cat_df.loc[mask, ['div', 'category1', 'category2']] = [row['div'], row['category1'], row['category2']]

    # 2. other_sku.json을 사용한 추가 분류 (map 활용으로 성능 개선)
    uncategorized_mask = cat_df['category1'].isna()
    if uncategorized_mask.any():
        print("2차 분류(SKU 리스트) 시작...")
        # 매핑을 위한 딕셔너리 생성 (대소문자 통일)
        sku_df['model_code'] = sku_df['model_code'].astype(str).str.upper()
        cat1_map = sku_df.set_index('model_code')['category1']
        cat2_map = sku_df.set_index('model_code')['category2']
        
        # map을 사용하여 분류 적용
        model_codes_to_map = cat_df.loc[uncategorized_mask, 'model_code'].str.upper()
        cat_df.loc[uncategorized_mask, 'category1'] = model_codes_to_map.map(cat1_map)
        cat_df.loc[uncategorized_mask, 'category2'] = model_codes_to_map.map(cat2_map)
        # 'div'는 'category1'과 동일하게 채움
        cat_df.loc[uncategorized_mask, 'div'] = cat_df.loc[uncategorized_mask, 'category1']

    print("카테고리 분류 완료.")
    return cat_df.dropna(subset=['category1'])

### 3. 분석 실행
정의된 함수들을 순서대로 실행하여 분석 진행

#### 3.1. 데이터 로딩 및 전처리

In [ ]:
# 1. 원본 데이터 로드
raw_df = load_raw_data(RAW_DATA_DIR)

# 2. 데이터 전처리
preprocessed_df = preprocess_data(raw_df)

# 3. 모델 코드 분리
basket_df = split_model_codes(preprocessed_df)

print("
전처리 및 모델 코드 분리 후 데이터 샘플:")
basket_df.head()

#### 3.2. 상품 카테고리 분류

In [ ]:
# 카테고리 분류 기준 정보 로드
category_regex_df = pd.read_json(CATEGORY_REGEX_PATH)
other_sku_df = pd.read_json(OTHER_SKU_PATH)

# 고유 모델 코드 추출
unique_codes = basket_df['model_code'].unique()
print(f"분석할 고유 모델 코드 수: {len(unique_codes):,}")

# 카테고리 분류 실행
categorized_codes_df = categorize_model_codes(pd.Series(unique_codes), category_regex_df, other_sku_df)

print(f"
분류된 모델 코드 수: {len(categorized_codes_df):,}")
print(f"미분류 모델 코드 수: {len(unique_codes) - len(categorized_codes_df):,}")

categorized_codes_df.head()

#### 3.3. 최종 데이터프레임 생성

In [ ]:
# 분류된 카테고리 정보를 원래 데이터에 병합 (left join)
final_df = pd.merge(basket_df, categorized_codes_df, on='model_code', how='left')

# '기타 배제' 카테고리는 분석에서 제외
final_df = final_df[final_df['category1'] != '기타 배제'].copy()

# 메모리 효율을 위해 불필요한 컬럼 제거
final_df = final_df[['order_id', 'date', 'model_code', 'category1', 'category2']]

print("최종 분석용 데이터프레임 정보:")
final_df.info()
print("
최종 데이터프레임 샘플:")
final_df.head()

### 4. 장바구니 분석 (연관 규칙 탐사)
Apriori 알고리즘을 사용하여 상품 간의 연관성을 분석

In [ ]:
# 분석을 위한 데이터 구조 생성 (order_id 기준, category2로 상품 목록 생성)
# dropna()를 통해 카테고리가 없는 상품은 제외
basket_lists = final_df.dropna(subset=['category2']).groupby('order_id')['category2'].apply(list)

# mlxtend 라이브러리 입력 형식(Transaction Matrix)으로 변환
te = TransactionEncoder()
te_ary = te.fit(basket_lists).transform(basket_lists)
transaction_df = pd.DataFrame(te_ary, columns=te.columns_)

print("Transaction Matrix 샘플:")
transaction_df.head()

In [ ]:
# Apriori 알고리즘으로 빈발 아이템셋 탐색
# min_support: 최소 지지도 (전체 거래 중 해당 상품(조합)이 포함된 거래의 비율)
frequent_itemsets = apriori(transaction_df, min_support=0.001, use_colnames=True)

# 연관 규칙 생성
# metric: 규칙 평가 척도 (lift, confidence 등)
# min_threshold: 해당 척도의 최소값
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

print("
생성된 연관 규칙 수:", len(rules))

#### 4.1. 분석 결과 확인

In [ ]:
print("빈발 아이템셋 (상위 5개):")
frequent_itemsets.sort_values('support', ascending=False).head()

In [ ]:
print("연관 규칙 (lift 기준 상위 10개):")
rules.sort_values('lift', ascending=False).head(10)